So now we have created all the tables, we need to make the SQLite table structure and then populate it. Let's start simple (no constraints) and add them in afterwards.

In [1]:
#Import libraries...

import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [149]:
#Location where existing CSVs from last step are stored.
path = r'C:\\Users\\Killian\\Projects\\SQL\\Superstore\\tables\\'

In [ ]:
# def create_connection(db_file):
#     """ create a database connection to a SQLite database """
#     conn = None
#     try:
#         conn = sqlite3.connect(db_file)
#         print(sqlite3.version)
#     except Error as e:
#         print(e)
#     finally:
#         if conn:
#             conn.close()


# if __name__ == '__main__':
#     create_connection(r"C:\sqlite\db\pythonsqlite.db")

In [150]:
#Creating an empty sqlite database to populate with the newly created dataframes
sales_db = sqlite3.connect("superstore_database.db")

#Alternate method
# import pyodbc 
# sales_db = pyodbc.connect('Driver={SQL Server};'
#                      'Server=KILLBOT-LAPTOP\SQLEXPRESS;'
#                      'Database=test_database;'
#                      'Trusted_Connection=yes;')

In [151]:
# Establishing a cursor object (a tool that executes SQL code against the database)
c = sales_db.cursor()

In [152]:
#create the (empty) tables which will be included in the database

#Customers
c.execute('''
CREATE TABLE IF NOT EXISTS "Customers" (
	"CustomerID"	CHAR(8),
	"CustomerName"	VARCHAR(30),
	"Segment"	VARCHAR(15),
	PRIMARY KEY("CustomerID")
);
''')

#Address
c.execute('''
CREATE TABLE IF NOT EXISTS "Address" (
	"CustomerID"	CHAR(8),
	"Region"	VARCHAR(7),
	"State"	VARCHAR(20),
	"City"	VARCHAR(30),
	"PostalCode"	MEDIUMINT,
	"AddressID"	CHAR(7),
	PRIMARY KEY("AddressID"),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

#Orders
c.execute('''
CREATE TABLE IF NOT EXISTS "Orders" (
	"OrderID"	CHAR(14),
	"CustomerID"	CHAR(8),
	"OrderDate"	DATE,
	"AddressID"	CHAR(7),
    "ShipDate"	DATE,
    "ShipMode"	VARCHAR(20),
	PRIMARY KEY("OrderID"),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

#Category
c.execute('''
CREATE TABLE IF NOT EXISTS "Category" (
	"Category"	VARCHAR(20),
	"Sub-Category"	VARCHAR(20),
	"CategoryID"	CHAR(5),
	PRIMARY KEY("CategoryID")
);
''')


#Products
c.execute('''
CREATE TABLE IF NOT EXISTS "Products" (
	"ProductID"	CHAR(15),
	"ProductName"	VARCHAR(200),
	"Catalogueprice"	MEDIUMINT,
	"Supplierprice"	MEDIUMINT,
	"CategoryID"	CHAR(5),
	PRIMARY KEY("ProductID"),
    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID)
);
''')


#Order Details
c.execute('''
CREATE TABLE IF NOT EXISTS "OrderDetails" (
	"OrderID"	CHAR(14),
	"ProductID"	CHAR(15),
	"Quantity"	TINYINT,
	"Sales"	INT,
	"Discount"	INT,
	"Profit"	INT,
	PRIMARY KEY("OrderID","ProductID"),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID)
);
''')

Now let's populate the empty tables. First step is converting the CSVs created in the previous notebook.

In [153]:
csv_list = [os.path.splitext(filename)[0] for filename in os.listdir(path)]

In [154]:
tables = [pd.read_csv(f'{path}{name}.csv') for name in csv_list]

In [155]:
table_dict = dict(zip(csv_list, tables))

In [156]:
table_list = iter(table_dict.keys())
table_contents = iter(table_dict.values())

for i in range (len(table_dict)):
    next(table_contents).to_sql((next(table_list)), sales_db, if_exists='replace', index=True)

In [ ]:
# 1. What is the category generating the maximum sales revenue?

# 2. What about the profit in this category?

# 3. Are they making a loss in any categories?

# 4. What are 5 states generating the maximum and minimum sales revenue?

# 5. What are the 3 products in each product segment with the highest sales?

# 6. Are they the 3 most profitable products as well?

# 7. What are the 3 best-seller products in each product segment? (Quantity-wise)

# 8. What are the top 3 worst-selling products in every category? (Quantity-wise)

# 9. How many unique customers per month are there for the year 2016

In [157]:
#Always best practice to close the connection once you're done.
sales_db.commit()
sales_db.close()